In [1]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime

In [2]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [3]:

# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,  Name, Nombre_Empresa__c from Programas_por_Empresas__c
    """
)
programas_empresas = pd.DataFrame(results["records"])
programas_empresas = programas_empresas[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Name", "Nombre_Empresa__c", "Empresa_Padre__c"]
]
programas_empresas.shape

(7880, 6)

In [4]:
# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select Id, Name, IdentificationId__pc, Identification_Type__c,Holding__c ,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,poliza__c,Empleador__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    ["Id", "Name", "IdentificationId__pc", "Identification_Type__c", "Holding__c", "Pa_s_del_Paciente__c",'Campa_a__c','AccountNumber','Id_Empresa__c','Empleador__c','poliza__c']
]
Empresas_SF.shape

(7875, 11)

In [5]:
Empresas_SF_grande = pd.merge(
    left=programas_empresas,
    right=Empresas_SF,
    how="left",
    left_on="Empresa__c",
    right_on="Id",
)
Empresas_SF_grande.head()

,Id_x,Empresa__c,Empresa_Padre__c,Name_x,Nombre_Empresa__c,Empresa_Padre__c,Id_y,Name_y,IdentificationId__pc,Identification_Type__c,Holding__c,Pa_s_del_Paciente__c,Campa_a__c,AccountNumber,Id_Empresa__c,Empleador__c,poliza__c
0,a1W8c000007vDN8EAM,0018c00002axQXrAAM,Metlife,A-0000,Metlife Holding Metlife Laboratorio Maver Ltda.,Metlife,0018c00002axQXrAAM,Metlife Holding Metlife Laboratorio Maver Ltda.,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5813,None,None
1,a1W8c000007vDN9EAM,0018c00002axQXsAAM,Metlife,A-0001,Metlife Holding Metlife Aguas Del Valle S.A.,Metlife,0018c00002axQXsAAM,Metlife Holding Metlife Aguas Del Valle S.A.,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5814,None,None
2,a1W8c000007vDNAEA2,0018c00002axQXtAAM,Metlife,A-0002,Metlife Holding Metlife Berliam Spa.,Metlife,0018c00002axQXtAAM,Metlife Holding Metlife Berliam Spa.,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5815,None,None
3,a1W8c000007vDNBEA2,0018c00002axQXuAAM,Metlife,A-0003,Metlife Holding Metlife Besalco Maquinarias S.A.,Metlife,0018c00002axQXuAAM,Metlife Holding Metlife Besalco Maquinarias S.A.,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5816,None,None
4,a1W8c000007vDNCEA2,0018c00002axQXvAAM,Metlife,A-0004,Metlife Holding Metlife Carlos Escarate Y Cia ...,Metlife,0018c00002axQXvAAM,Metlife Holding Metlife Carlos Escarate Y Cia ...,None,None,Holding Metlife,a1U8c000005dAtnEAE,Metlife,None,EMP5817,None,None


In [6]:
results = sf.query_all(
    """
   select Id,Paciente__c,Programas_por_Empresas__c from Paciente_Programas__c
    """
)
pacientes_programas = pd.DataFrame(results["records"])
pacientes_programas = pacientes_programas[
    ["Id", "Paciente__c", "Programas_por_Empresas__c"]
]
pacientes_programas.shape

(1220144, 3)

In [7]:
# recuperar todos los pacientes ID
results = sf.query_all(
    """
   select Id, IdentificationId__pc,Name from Account 
    """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[["Id", "IdentificationId__pc", "Name"]]
Pacientes_SF.shape

(1540195, 3)

In [8]:
programaxempresa = pd.merge(
    left=pacientes_programas,
    right=Pacientes_SF,
    how="left",
    left_on="Paciente__c",
    right_on="Id",
)
programaxempresa.shape

(1220144, 6)

In [9]:
programaxempresa.head()

,Id_x,Paciente__c,Programas_por_Empresas__c,Id_y,IdentificationId__pc,Name
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,0018c00002amLbpAAE,224488556,Genesis Antonia Gutierrez Abarca
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,0018c00002amLbqAAE,113376066,Claudia Silva
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,0018c00002amLbrAAE,79476161,Maria Lorenza Garay
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,0018c00002amLbsAAE,181854227,Estolaza Vidal Jean Patrick Moises
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,0018c00002amLc2AAE,126899823,Rivas Correa Claudia


In [11]:
nuevos = ("F:/Empresas SF/EY/activo.xlsx")
Pacientes_Archivo = pd.read_excel(nuevos)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_5092\1838615774.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  Pacientes_Archivo = pd.read_excel(nuevos)


In [12]:
archivoempresa = pd.merge(
    left=Pacientes_Archivo,
    right=programaxempresa,
    how="left",
    left_on="rut1",
    right_on="IdentificationId__pc",
)
archivoempresa.shape

(3036, 16)

In [13]:
archivoempresa.to_excel(
    "F:/Empresas SF/EY/Pacientes_y_empresas_nuevas2.xlsx", index=False
)

In [14]:
archivoempresa_grande = pd.merge(
    left=archivoempresa,
    right=Empresas_SF_grande,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id_x",
)
archivoempresa_grande.shape

(3036, 33)

In [15]:
archivoempresa_grande.to_excel(
    "F:/Empresas SF/EY/Pacientes_y_empresas_nuevas_total.xlsx", index=False
)

In [16]:
nuevos = ("F:/Empresas SF/EY/inactivo.xlsx")
Pacientes_Archivo = pd.read_excel(nuevos)
archivoempresa = pd.merge(
    left=Pacientes_Archivo,
    right=programaxempresa,
    how="left",
    left_on="rut1",
    right_on="IdentificationId__pc",
)
archivoempresa.shape

(644, 16)

In [17]:
archivoempresa_grande = pd.merge(
    left=archivoempresa,
    right=Empresas_SF_grande,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id_x",
)
archivoempresa_grande.shape

(644, 33)

In [18]:
archivoempresa_grande.to_excel(
    "F:/Empresas SF/EY/Pacientes_y_empresas_nuevas_total_inactivo.xlsx", index=False
)

In [ ]:
archivoempresa_grande = pd.merge(
    left=archivoempresa_grande,
    right=Pacientes_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id_x",
)
archivoempresa_grande.shape